In [1]:
from tensorflow import keras
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import json
import os
pd.set_option('display.max_colwidth', None)

In [2]:
# function to load training & test sets into DataFrames
def load_data(filename : str):
    meta = pd.read_csv(filename).reset_index()
    actors = meta['actor_name']
    paths = meta['path']
    meta = np.vstack((actors, paths))
    meta_df = pd.DataFrame(meta).T
    meta_df.columns = ['actor_name', 'path']
    return meta_df

# predict a class using img file
def predict_class(filepath):
    import cv2
    img = cv2.imread(filepath)
    resized_img = cv2.resize(img, (128, 128))
    normalizedImg = np.zeros((800, 800))
    normalizedImg = cv2.normalize(resized_img, normalizedImg, 0, 255, cv2.NORM_MINMAX)
    resized_img = normalizedImg.reshape(1, 128, 128, 3)
    return lookup_table[str(np.argmax(model.predict(resized_img)))]

def evaluate_model(test_set, lookup_table):
    # set the truth list and convert to ints
    truth = test_set['actor_name']
    name_to_int = dict(map(reversed, lookup_table.items()))
    truth = [name_to_int[name] for name in truth]
    
    # create paths of images to predict
    images = test_set['path']
    images = [path.replace('.\data\lfw-deepfunneled', '../../../data/lfw-deepfunneled') for path in images]
    
    predictions = []
    for image in images:
        predictions.append(name_to_int[predict_class(image)])
        
    return classification_report(truth, predictions, labels=np.unique(predictions))

In [3]:
## load data of model
model = keras.models.load_model('8_epoch25.h5')
train_set = load_data(r'data/train_set.csv')
test_set = load_data(r'data/test_set.csv')
lookup_table = {}
with open('data/lookup_table.json', 'r') as f:
    lookup_table = json.loads(f.read())
# lookup_table2 = json.loads(lookup_table)

In [108]:
model.summary()

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_134 (Conv2D)         (None, 126, 126, 8)       224       
                                                                 
 max_pooling2d_134 (MaxPooli  (None, 63, 63, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_135 (Conv2D)         (None, 61, 61, 16)        1168      
                                                                 
 max_pooling2d_135 (MaxPooli  (None, 30, 30, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_136 (Conv2D)         (None, 28, 28, 32)        4640      
                                                                 
 max_pooling2d_136 (MaxPooli  (None, 14, 14, 32)     

In [111]:
report = evaluate_model(test_set, lookup_table)

1/1 [==============================] - 0s 41ms/step


In [114]:
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         3
         101       0.07      0.50      0.13         6
         102       0.00      0.00      0.00         4
         103       0.00      0.00      0.00         8
         108       0.04      0.33      0.08         3
         109       0.50      0.10      0.17        10
          11       0.12      0.27      0.17        15
         110       0.00      0.00      0.00         3
         111       0.00      0.00      0.00         6
         113       0.25      0.14      0.18         7
         114       0.00      0.00      0.00         3
         115       0.33      0.25      0.29         4
         116       0.00      0.00      0.00         2
         117       0.00      0.00      0.00         4
         119       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         8
         120       0.00    